In [1]:
import mysql.connector as sql
import yaml
import ujson as js
import tqdm
from os import walk

In [2]:
class RegionUpload(object):
    root_path = 'sde/fsd/universe'
    file_name = 'region.staticdata'
    sql_settings_path = 'database_login.json'
    sql_table = 'Regions'
    sql_cols = (
        'region_id', 'region_name', 'name_id', 'desc_id',
        'nebula_id', 'wormhole_class_id', 'min_x', 'min_y',
        'min_z', 'center_x', 'center_y', 'center_z',
        'max_x', 'max_y', 'max_z'
    )
    sql_scripts = {
        'select': {
            'name': 'SELECT object_name FROM ObjectNames WHERE object_name_id = %s;'
        },
        'insert': {
            'region': 'INSERT INTO %s (%s) VALUES (%s);'
        }
    }
    
    def __init__(self, start_active = True):
        if start_active:
            self.run_process()
    
    def run_process(self):
        self.connect_sql()
        self.parse_regions()
        self.close_sql()
    
    def connect_sql(self):
        
        def build_sql_conn(file_path):
            with open(file_path) as sql_file:
                sql_conn_params = js.load(sql_file)

            sql_conn = sql.connect(**sql_conn_params)
            return sql_conn
        
        self.sql_conn = build_sql_conn(self.sql_settings_path)
        self.sql_cur = self.sql_conn.cursor()
        
    def parse_regions(self):
        for dir_path, _, files in walk(self.root_path):
            if self.file_name in files:
                self.process_region(dir_path)
    
    def process_region(self, dir_path):
        file_data = self.load_file(dir_path, self.file_name)
        parsed_data = self.parse_data(file_data)
        self.insert_data(parsed_data)
                
    def load_file(self, file_path, file_name):
        with open('%s/%s' % (file_path, file_name)) as file_obj:
            file_data = yaml.load(file_obj)
            
        return file_data
    
    def get_name(self, name_id):
        self.sql_cur.execute(self.sql_scripts['select']['name'], (name_id,))
        obj_name = self.sql_cur.fetchall()[0][0]
        return obj_name
    
    def parse_data(self, file_data):
        parsed_data = (
            file_data['regionID'],
            self.get_name(file_data['regionID']),
            file_data.get('nameID', None),
            file_data.get('descriptionID', None),
            file_data.get('nebula', None),
            file_data.get('wormholeClassID', None),
            *file_data.get('min', []),
            *file_data.get('center', []),
            *file_data.get('max', [])
        )
        return parsed_data
    
    def insert_data(self, parsed_data):
        sql_script = self.build_sql_script()
        self.sql_cur.execute(sql_script, parsed_data)
        self.sql_conn.commit()
    
    def build_sql_script(self):
        sql_script = self.sql_scripts['insert']['region'] % (
            self.sql_table,
            ','.join(self.sql_cols),
            ','.join(('%s',) * len(self.sql_cols))
        )
        return sql_script
    
    def close_sql(self):
        self.sql_cur.close()
        self.sql_conn.close()

In [3]:
class ConstUpload(object):
    root_path = 'sde/fsd/universe'
    file_name = 'constellation.staticdata'
    sql_settings_path = 'database_login.json'
    sql_table = 'Constellations'
    sql_cols = (
        'constellation_id', 'region_id', 'constellation_name', 'name_id',
        'faction_id', 'wormhole_class_id', 'min_x', 'min_y',
        'min_z', 'center_x', 'center_y', 'center_z',
        'max_x', 'max_y', 'max_z'
    )
    sql_scripts = {
        'select': {
            'name': 'SELECT object_name FROM ObjectNames WHERE object_name_id = %s;'
        },
        'insert': {
            'region': 'INSERT INTO %s (%s) VALUES (%s);'
        }
    }
    
    def __init__(self, start_active = True):
        if start_active:
            self.run_process()
    
    def run_process(self):
        self.connect_sql()
        self.parse_data()
        self.close_sql()
    
    def connect_sql(self):
        
        def build_sql_conn(file_path):
            with open(file_path) as sql_file:
                sql_conn_params = js.load(sql_file)

            sql_conn = sql.connect(**sql_conn_params)
            return sql_conn
        
        self.sql_conn = build_sql_conn(self.sql_settings_path)
        self.sql_cur = self.sql_conn.cursor()
        
    def parse_data(self):
        for dir_path, _, files in walk(self.root_path):
            if self.file_name in files:
                self.region_id = self.get_region_id('/'.join(dir_path.split('/')[:-1]))
                self.process_data(dir_path)
                
    def get_region_id(self, region_path):
        region_data = self.load_file(region_path, 'region.staticdata')
        return region_data['regionID']
    
    def process_data(self, dir_path):
        file_data = self.load_file(dir_path, self.file_name)
        parsed_data = self.parsing_data(file_data)
        self.insert_data(parsed_data)
                
    def load_file(self, file_path, file_name):
        with open('%s/%s' % (file_path, file_name)) as file_obj:
            file_data = yaml.load(file_obj)
            
        return file_data
    
    def parsing_data(self, file_data):
        parsed_data = (
            file_data['constellationID'],
            self.region_id,
            self.get_name(file_data['constellationID']),
            file_data.get('nameID', None),
            file_data.get('factionID', None),
            file_data.get('wormholeClassID', None),
            *file_data.get('min', []),
            *file_data.get('center', []),
            *file_data.get('max', [])
        )
        return parsed_data
    
    def get_name(self, name_id):
        self.sql_cur.execute(self.sql_scripts['select']['name'], (name_id,))
        obj_name = self.sql_cur.fetchall()[0][0]
        return obj_name
    
    def insert_data(self, parsed_data):
        sql_script = self.build_sql_script()
        self.sql_cur.execute(sql_script, parsed_data)
        self.sql_conn.commit()
    
    def build_sql_script(self):
        sql_script = self.sql_scripts['insert']['region'] % (
            self.sql_table,
            ','.join(self.sql_cols),
            ','.join(('%s',) * len(self.sql_cols))
        )
        return sql_script
    
    def close_sql(self):
        self.sql_cur.close()
        self.sql_conn.close()

In [4]:
class SystemUpload(object):
    root_path = 'sde/fsd/universe'
    file_name = 'solarsystem.staticdata'
    sql_settings_path = 'database_login.json'
    sql_table = 'Systems'
    sql_cols = (
        'system_id', 'constellation_id', 'system_name', 'name_id',
        'desc_id', 'faction_id', 'wormhole_class_id',
        'border', 'corridor', 'fringe', 'hub', 'international', 'regional',
        'luminosity', 'radius', 'security', 'security_class', 'visual_effect',
        'min_x', 'min_y', 'min_z', 'center_x', 'center_y', 'center_z', 'max_x', 'max_y', 'max_z'
    )
    sql_scripts = {
        'select': {
            'name': 'SELECT object_name FROM ObjectNames WHERE object_name_id = %s;'
        },
        'insert': {
            'region': 'INSERT INTO %s (%s) VALUES (%s);'
        }
    }
    
    def __init__(self, start_active = True):
        if start_active:
            self.run_process()
    
    def run_process(self):
        self.connect_sql()
        self.parse_data()
        self.close_sql()
    
    def connect_sql(self):
        
        def build_sql_conn(file_path):
            with open(file_path) as sql_file:
                sql_conn_params = js.load(sql_file)

            sql_conn = sql.connect(**sql_conn_params)
            return sql_conn
        
        self.sql_conn = build_sql_conn(self.sql_settings_path)
        self.sql_cur = self.sql_conn.cursor()
        
    def parse_data(self):
        for dir_path, _, files in tqdm.tqdm(walk(self.root_path)):
            if self.file_name in files:
                self.constellation_id = self.get_const_id('/'.join(dir_path.split('/')[:-1]))
                self.process_data(dir_path)
                
    def get_const_id(self, region_path):
        constellation_id = self.load_file(region_path, 'constellation.staticdata')
        return constellation_id['constellationID']
    
    def process_data(self, dir_path):
        file_data = self.load_file(dir_path, self.file_name)
        parsed_data, stargate_data = self.parsing_data(file_data)
        self.insert_data(parsed_data)
        self.insert_stargates(stargate_data)
                
    def load_file(self, file_path, file_name):
        with open('%s/%s' % (file_path, file_name)) as file_obj:
            file_data = yaml.load(file_obj)
            
        return file_data
    
    def parsing_data(self, file_data):
        parsed_data = (
            file_data['solarSystemID'],
            self.constellation_id,
            self.get_name(file_data['solarSystemID']),
            file_data.get('solarSystemNameID', None),
            file_data.get('descriptionID', None),
            file_data.get('factionID', None),
            file_data.get('wormholeClassID', None),
            file_data.get('border', None),
            file_data.get('corridor', None),
            file_data.get('fringe', None),
            file_data.get('hub', None),
            file_data.get('international', None),
            file_data.get('regional', None),
            file_data.get('luminosity', None),
            file_data.get('radius', None),
            file_data.get('security', None),
            file_data.get('securityClass', None),
            file_data.get('visualEffect', None),
            *file_data.get('min', [None] * 3),
            *file_data.get('center', [None] * 3),
            *file_data.get('max', [None] * 3)
        )
        stargate_data = []
        for key, value in file_data.get('stargates', {}).items():
            stargate_item = (
                key,
                file_data['solarSystemID'],
                value.get('destination', None),
                value.get('typeID', None),
                *value.get('position', [None] * 3)
            )
            stargate_data.append(stargate_item)
        return (parsed_data, stargate_data)
    
    def get_name(self, name_id):
        self.sql_cur.execute(self.sql_scripts['select']['name'], (name_id,))
        obj_name = self.sql_cur.fetchall()[0][0]
        return obj_name
    
    def insert_data(self, parsed_data):
        sql_script = self.build_sql_script()
        try:
            self.sql_cur.execute(sql_script, parsed_data)
        except:
            print(parsed_data)
            raise Exception('splat')
        self.sql_conn.commit()
        
    def insert_stargates(self, stargate_data):
        sql_script = 'INSERT INTO %s (%s) VALUES (%s);' % (
            'Stargates',
            ','.join(('stargate_id', 'system_id', 'dest_stargate_id', 'type_id', 'pos_x', 'pos_y', 'pos_z')),
            ','.join(('%s',) * 7)
        )
        self.sql_cur.executemany(sql_script, stargate_data)
        self.sql_conn.commit()
    
    def build_sql_script(self):
        sql_script = self.sql_scripts['insert']['region'] % (
            self.sql_table,
            ','.join(self.sql_cols),
            ','.join(('%s',) * len(self.sql_cols))
        )
        return sql_script
    
    def close_sql(self):
        self.sql_cur.close()
        self.sql_conn.close()

In [5]:
RegionUpload()
#ConstUpload()
#SystemUpload()

DataError: 1264 (22003): Out of range value for column 'min_x' at row 1